In [1]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/Ad/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["fsp_2e3","fsp_3e3","fsp_4e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


2018-09-07 15:30:11.491862: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:30:11.491879: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:30:11.491882: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-07 15:30:11.491885: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
07-Sep 15:30:11:WARNING:root:replacing docs for 'close :: Tuple{ZMQTransport}' in module 'Main'.
07-Sep 15:30:11

run_env_server_rarl (generic function with 1 method)

In [4]:
mutable struct Counter
    i::Int
    I::Int
    num_collision::Int
    rate_collision::Float64
    total_reward::Float64
    single_reward::Float64
    avg_reward::Float64
    max_reward::Float64
    min_reward::Float64
    rewards::Array{Float64}
    done::Bool
    direction::Int
    total_speed::Float64
    single_speed::Float64
    avg_speed::Float64
    speeds::Array{Float64}
    single_time::Float64
    total_time::Float64
    time_between_collisions::Float64
    total_reward_d::Float64
    single_reward_d::Float64
    avg_reward_d::Float64
    rewards_d::Array{Float64}
end
function Counter(N::Int)
    return Counter(0,N,0,0.0,0.0,0.0,0.0,-Inf,Inf,zeros(N),false,-1,
        0.0,0.0,0.0,zeros(N),0.0,0.0,0.0,0.0,0.0,0.0,zeros(N))
end

Counter

In [5]:
function handle_action_ad!(pc,scene,models,roadway,counter,rarl)
    state = get_observation(scene,models,roadway)
    
    action1_arr, action2_arr = pc[:getAction](state)
    action1_arr = tanh.(action1_arr)
    action2_arr = tanh.(action2_arr)

    action = Egoaction(action1_arr[1],action1_arr[2],
                       action2_arr[1],action2_arr[2])
    
    done, bump = simulate_action!(action,scene,models,roadway)
    counter.single_time += TIMESTEP
    counter.total_time += TIMESTEP
    reward = reward_fn(action,done,scene,models,roadway)
    if bump
        counter.num_collision += 1
    else
        counter.total_reward_d += reward
        counter.single_reward_d += reward
    end
    
    counter.total_reward += reward
    counter.single_reward += reward
    
    speed = scene[1].state.v
    counter.single_speed += speed
    counter.total_speed += speed
    
    if done
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.single_reward > counter.max_reward
            counter.max_reward = counter.single_reward
        end
        if counter.single_reward < counter.min_reward
            counter.min_reward = counter.single_reward
        end
        if counter.i > counter.I
            counter.done = true
            counter.avg_reward = counter.total_reward/counter.I
            counter.rate_collision = counter.num_collision/counter.I
            counter.avg_reward_d = counter.total_reward_d/counter.I
            counter.avg_speed = counter.total_speed/(counter.total_time/TIMESTEP)
            counter.time_between_collisions = counter.total_time/counter.num_collision
        else
            counter.rewards[counter.i] = counter.single_reward
            counter.rewards_d[counter.i] = counter.single_reward_d
            counter.speeds[counter.i] = counter.single_speed/(counter.single_time/TIMESTEP)
        end
        counter.single_reward = 0.0
        counter.single_reward_d = 0.0
        counter.single_speed = 0.0
        counter.single_time = 0.0
    end
    
    
end

handle_action_ad! (generic function with 1 method)

In [6]:
scene, models, roadway = initialize_env()
log_path = string(path,"Data/")
for pname in pnames
    println(pname)
#     files = readdir(string(path,"Policy/",pname))
#     itrs = []
#     for i = 1:length(files)
#         if files[i] != ".DS_Store"
#             push!(itrs,policies[i][1:length(files[i])-4])
#         end
#     end
    
    for alpha in 0:10:200
        println(alpha)
        policy_path = string(path,"Policy/",pname,"/itr_",alpha,".pkl")
        pc[:reset_policy](policy_path=policy_path)
        N = 200

        counter = Counter(N)
        rand_ego!(scene,models,roadway)
        pc[:resetPolicy](1)
        while !counter.done
            handle_action_ad!(pc,scene,models,roadway,counter,true)
        end
        println(counter.avg_reward)
        println(counter.rate_collision)

        writedlm(string(log_path,pname,"_rewards_ad",alpha,".txt"),counter.rewards)
        writedlm(string(log_path,pname,"_rewards_d_ad",alpha,".txt"),counter.rewards_d)
        writedlm(string(log_path,pname,"_speeds_ad",alpha,".txt"),counter.speeds)
        writedlm(string(log_path,pname,"_collision_rate_ad",alpha,".txt"),counter.rate_collision)
#         writedlm(string(log_path,pname,"_time_between_collisions_pareto",alpha,".txt"),(counter.total_time+counter2.total_time)/(counter.num_collision+counter2.num_collision))
    end
end

fsp_2e3
0
515.7066937866733
0.0
10
518.1213722403346
0.0
20
520.6111041474198
0.0
30
522.5246503193481
0.0
40
517.495979297116
0.0
50
520.4454125954549
0.0
60
514.9647372627752
0.0
70
515.5286446739225
0.0
80
518.8808021194415
0.0
90
517.4017582549002
0.0
100
518.5680685620569
0.0
110
515.1287605611288
0.0
120
519.8196492623237
0.0
130
517.6725876462526
0.0
140
514.4640032021321
0.0
150
515.6125417902483
0.0
160
520.6773238956206
0.0
170
516.4126529390056
0.0
180
517.6873802086927
0.0
190
517.9007040004816
0.0
200
516.7658327107453
0.0
fsp_3e3
0
550.8138744348897
0.0
10
551.913761004294
0.0
20
546.1268661067065
0.0
30
549.7565849283072
0.0
40
550.089841926253
0.0
50
549.2167564099225
0.0
60
549.506917434617
0.0
70
548.8939527281173
0.0
80
550.8658191340871
0.0
90
549.4923190569369
0.0
100
548.7904336681405
0.0
110
550.979939389084
0.0
120
549.5894812436466
0.0
130
550.8530721343626
0.0
140
551.9016494169631
0.0
150
550.8352602164333
0.0
160
546.7842571335888
0.0
170
550.868960409477
0.